## DSPy Intro Series 01 - Basic Prompting

Get started with DSPy!

Read the blog:

https://publish.obsidian.md/infinite-loop/blog/dspy-intro/DSPy+Intro+Series+01

### Init

In [ ]:
# Install & import stuff
%%capture
!pip install dspy-ai
!pip install arize-phoenix openinference-instrumentation-dspy opentelemetry-exporter-otlp

import os
from getpass import getpass

import dspy
import openai
import phoenix as px

In [ ]:
# Logging
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

phoenix_session = px.launch_app()
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [ ]:
# Check for needed user dara
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

### Set Up

Connect dspy with model

In [ ]:
import dspy

llm = dspy.OpenAI(
    model='gpt-3.5-turbo',
    api_key=os.getenv("OPENAI_API_KEY"),
    max_tokens=4000
)

dspy.settings.configure(lm=llm)

### Signatures

Try out different signatures!

In [ ]:
predictor = dspy.Predict("question -> answer")
print(predictor(question="What is a transformer model?"))

In [ ]:
predictor = dspy.Predict("idea -> projectplan")
print(predictor(idea="A webshop with flask"))

In [ ]:
predictor = dspy.Predict("subject -> title, subtitle, blog_outline")
print(predictor(subject="Cats"))

### Signature class

Implement the signature as a class!

In [ ]:
class BasicBlogGenerator(dspy.Signature):
    subject = dspy.InputField()
    title = dspy.OutputField()
    subtitles = dspy.OutputField()
    blog_outline = dspy.OutputField()

generate_blog = dspy.Predict(BasicBlogGenerator)

subject='Cats (use dry-humor and sarcasm)'
pred = generate_blog(subject=subject)

print(f"Subject:\n{subject}")
print(f"Title:\n{pred.title}")
print(f"Subtitles:\n{pred.subtitles}")
print(f"Outline:\n{pred.blog_outline}")

### Logs

Take a look what's happening in the background!

In [ ]:
print(phoenix_session.url)